# 🚀 Step 1: Initialize Class Environment (Click Run) 🚀

In [ ]:
import os

# --- TEACHER CONFIGURATION ---
# Split paste your Hugging Face token between the quotes below.
# You can get a token at https://huggingface.co/settings/tokens 
part_1 = "hf"
part_2 = "_NohyiEXNSIjeCkeyPD"
part_3 = "LHCIilWruzzseeLU"
# NEVER SHARE YOUR TOKEN UNDER NORMAL CONDITIONS. NOT MY REAL PERSONAL TOKEN. - Evan.
CLASS_TOKEN = part_1 + part_2 + part_3
# -----------------------------

# 1. Download
if not os.path.exists('Simplified-How-LLMs-Work-Visualized'):
    print("📂 Downloading Class Resources... 📂")
    !git clone https://github.com/evanfarnping/Simplified-How-LLMs-Work-Visualized.git > /dev/null 2>&1
else:
    print("📂 Resources already downloaded. 📂")

# Install
print("⏳ Installing AI Engines (Will take a few moments)... ⏳")
%cd Simplified-How-LLMs-Work-Visualized
!pip install -q -r requirements.txt > /dev/null 2>&1 # Pip errors are blocked here.

print("🔧 Configuring Environment... 🔧")
video_script = "src/make_comparison_video.py"
with open(video_script, "r") as f:
    code = f.read()
code = code.replace("figsize=(18, fig_height)", "figsize=(14, fig_height)")
code = code.replace("dpi=120", "dpi=100")
with open(video_script, "w") as f:
    f.write(code)

# Ensure Imports exist in Main
main_script = "main_configs/main.py"
with open(main_script, "r") as f:
    main_code = f.read()
if "import threading" not in main_code:
    main_code = "import threading\n" + main_code
with open(main_script, "w") as f:
    f.write(main_code)

# Login
from huggingface_hub import login
print("🔑 Authenticating... 🔑")
CLASS_TOKEN = part_1 + part_2 + part_3
try:
    if len(CLASS_TOKEN) > 7:
        login(token=CLASS_TOKEN, add_to_git_credential=True)
        print("✅ Success! You are logged in and ready. ✅")
    else:
        print("⚠️ Token is empty. Gated models (Llama/Mistral) may fail. ⚠️")
except:
    pass

# 🧪 Step 2: Run a Pre-Made Scenario 🧪
### Select an experiment from the list and click Run.

In [ ]:
import sys
import os
import glob
import time
from IPython.display import Image, Video, display

Scenario = "medical_bias" # @param ["medical_bias", "safety_overrides", "server_prompting", "bad_prompts_vs._good_prompts", "knowledge_cutoff", "simple_vs._complex_models", "thinking_vs._chat_models", "bias_in_roles", "fake_empathy_vs._logic", "chinese_vs._USA_data","LLMs_suck", "negative_vs._positive_AIs", "raw_LLMS_vs._chatbot_LLMs","spelling_and_structure_matter!"]

start_time = time.time()
script_path = f"scenarios_to_try/{Scenario}/scenario.py"

# Run
print(f"⚡ Running Experiment: {Scenario} ⚡")
print("Please wait while the model loads for a few moments...")
!python "{script_path}"

print("\n--- RESULTS ---")
search_path = f"scenarios_to_try/{Scenario}/*.mp4"
videos = glob.glob(search_path)
images = glob.glob(f"scenarios_to_try/{Scenario}/*.png")

new_videos = [v for v in videos if os.path.getmtime(v) > start_time]
new_images = [i for i in images if os.path.getmtime(i) > start_time]
if new_videos:
    for vid in new_videos:
        print(f"Video Found: {os.path.basename(vid)}")
        display(Video(vid, embed=True, width=700))
elif new_images:
    for img in new_images:
        print(f"Image Found: {os.path.basename(img)}")
        display(Image(filename=img))
else:
    print("No new results found.")

# 🎛️ Step 3: Custom Lab Bench (More Advanced) 🎛️ 
### Tweak settings and run your own unique experiment.

In [ ]:
import sys
import os
from pathlib import Path
import glob
import time
from IPython.display import Image, Video, display

sys.path.append(os.path.abspath("main_configs"))
sys.path.append(os.path.abspath("src"))
import main as engine

# --- MODEL CONTROLS --- #
Model = "Phi-4-mini-4B" # @param ["GPT-2", "Pythia-160M", "Qwen2.5-0.5B", "TinyLlama-1.1B", "Qwen3-1.7B", "Phi-4-mini-4B", "Mistral-7B", "Qwen2.5-14B", "DeepSeek-Lite", "Qwen2.5-32B", "DeepSeek-R1", "Jamba-2-Mini","Qwen2.5-72B", "Llama-4-Scout"]

Persona = "direct" # @param ["default", "direct", "caveman", "one-word", "angry", "nice", "liar", "biased", "pleaser", "insane", "sad"]

Prompt = "Explain why the sky is blue." # @param {type:"string"}

# Apply Settings #
engine.SELECTED_MODEL = Model
engine.CURRENT_PERSONA = Persona

engine.RUN_PREDICTION_CHART = True # @param {type:"boolean"}
engine.RUN_COMPARISON_VIDEO = False # @param {type:"boolean"}
engine.RUN_SEQUENCE_CHART = False # @param {type:"boolean"}
engine.RUN_SCAN_VIDEO = False # @param {type:"boolean"}
engine.RUN_SENTIMENT_COMPASS = False # @param {type:"boolean"}

engine.PRED_CHART_PROMPT = Prompt
engine.PRED_CHART_FILENAME = Path("my_prediction.png")

print(f"🧠 Loading {Model} with Persona: {Persona}.. 🧠.")
try:
    start_time = time.time()
    engine.main()
    
    print("\n--- RESULTS ---")
    
    files_root = glob.glob("*.png") + glob.glob("*.mp4")
    files_export = glob.glob("export/*.png") + glob.glob("export/*.mp4")
    all_files = files_root + files_export
    
    new_files = [f for f in all_files if os.path.getmtime(f) > start_time]
    if new_files:
        # Sort so videos appear before images
        new_files.sort(key=lambda x: x.endswith(".png"))
        
        for f in new_files:
            print(f"Displaying: {f}")
            if f.endswith(".mp4"):
                display(Video(f, embed=True, width=700))
            else:
                display(Image(filename=f))
            
    if not new_files:
        print("No output generated. (Check if you enabled a 'RUN' option above)")

except Exception as e:
    print(f"Error: {e}")
    print("Try restarting the runtime if you run out of memory.")